# Exercise: Vegetation change detection

## Overview

In this exercise, we will create a notebook to detect vegetation change. To compare change, we need to have data from two different times: an older dataset, and a newer dataset.

The notebook will include the following steps:

* Load Landsat 8 data
* Calculate a vegetation index for the loaded data
* Split the vegetation index data into two &mdash; an older half and a newer half
* Compute the mean composite for each half; and
* Compare the older and newer averages to check for vegetation change.

## Set up notebook

### Load packages and functions

In the first cell, type the following code and then run the cell to import necessary Python dependencies.

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

import sys
import datacube

sys.path.append('../../Scripts')
from deafrica_datahandling import load_ard
from deafrica_plotting import display_map
from deafrica_bandindices import calculate_indices

### Connect to the datacube

Enter the following code and run the cell to create our `dc` object, which provides access to the datacube.

In [3]:
dc = datacube.Datacube(app="Vegetation_exercise")

### Select area of interest

We need to select an area of interest. Some areas to check for vegetation changes include illegal mining sites (devegetation), and seasonal changes for crops (e.g. chlorophyll changes).

We will be selecting an area around a center point. Enter the following code and run the cell to select an area and time range of interest. The parameters are:
* `latitude`: The latitude at the centre of your area of interest.
* `longitude`: The longitude at the centre of your area of interest.
* `buffer`: The number of degrees to load around the central latitude and longitude.
* `time`: The date range to analyse. For reasonable results, the range should span at least two years to prevent detecting seasonal changes.
* `time_baseline`: The date at which to split the total sample into two non-overlapping samples. For this exercise, we choose a date halfway between `time[0]` and `time[1]`. Its value here, `'2015-12-31'`, is halfway between `'2013-01-01'` and `'2018-12-31'` - the time range in `time`. So our 2 time periods will be 2013 to 2015 and 2016 to 2018.

In the next cell, enter the following code, and then run it to select an area.

In [4]:
# Define the area of interest
latitude = 0.02
longitude = 35.425
buffer = 0.1

# Combine central lat,lon with buffer to get area of interest
lat_range = (latitude-buffer, latitude+buffer)
lon_range = (longitude-buffer, longitude+buffer)

# Set the range of dates for the complete sample
time = ('2013-01-01', '2018-12-01')

# Set the date to separate the data into two samples for comparison
time_baseline = '2015-12-31'

In the next cell, enter the following code, and then run it to show the area on a map.

In [5]:
display_map(x=lon_range, y=lat_range)

### Load data

In the new cell below, enter the following code, and then run it to load Landsat 8 data.

In [ ]:
landsat_ds = load_ard(
    dc=dc, products=["ls8_usgs_sr_scene"], 
    lat=lat_range, lon=lon_range,
    time=time, 
    output_crs="EPSG:6933",
    resolution=(-30, 30),
    align=(15, 15),
    group_by='solar_day',
    measurements=['nir', 'red', 'blue'],
    min_gooddata=0.7)

Using pixel quality parameters for USGS Collection 1
Finding datasets
    ls8_usgs_sr_scene
Counting good quality pixels for each time step


## Calculate indices

Now we need to calculate a vegetation index - or "vegetation proxy" - like NDVI. 

Until now, we have used NDVI, which uses the ratio of the red and near-infrared (NIR) bands to identify live green vegetation. 
The formula is

$$
\begin{aligned}
\text{NDVI} = \frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red}}.
\end{aligned}
$$

This time we will use EVI - the "Enhanced Vegetation Index", which uses the red, near-infrared (NIR) and blue bands to identify vegetation; it is particularly sensitive to high biomass regions.
The formula is

$$
\begin{aligned}
\text{EVI} = \frac{2.5 \times (\text{NIR} - \text{Red})}{\text{NIR} + 6 \times \text{Red} - 7.5 \times \text{Blue} + 1}.
\end{aligned}
$$

In the next cell, enter the following code, and then run it to calculate the EVI vegetation index for this data.

In [ ]:
landsat_ds = calculate_indices(landsat_ds, index='EVI', collection='c1')

## Detect changes

Now we need to determine the change in the vegetation index between the older and newer halves of the data.

First, we will split the vegetation index data into the older half and newer half.

In the next cell, enter the following code, and then run it to split the data.

In [ ]:
baseline_sample = landsat_ds.EVI.sel(time=slice(time[0], time_baseline))
postbaseline_sample = landsat_ds.EVI.sel(time=slice(time_baseline, time[1]))

### Detect per-pixel changes

Now we will calculate the differences in the average vegetation index between the halves for each pixel.

In the next cell, enter the following code, and then run it to create mean composites for the 2 time periods.

In [ ]:
baseline_composite = baseline_sample.mean('time')
postbaseline_composite = postbaseline_sample.mean('time')

Now we need to subtract the first time period EVI mean composite from the second time period EVI mean composite to determine the change in the vegetation index.

In the next cell, enter the following code, and then run it to determine the change in the vegetation index.

In [ ]:
diff_mean_composites = postbaseline_composite - baseline_composite

In the next cell, enter the following code, and then run it to show the difference between the mean composites for the time periods. This will allow us to see where the vegetation index increased or decreased and by how much.

In [ ]:
plt.figure(figsize=(9, 8))
diff_mean_composites.plot.imshow()
plt.title("Mean Composite Difference (Older to Newer)")
plt.show()

### Detect total change

Sometimes we want to know exactly how much the average vegetation index value changes between time periods - not just how much the average of the values for each pixel changes.

In the next cell, enter the following code, and then run it to compute the average of the vegetation index values for the older and newer halves. Note that the `.values` after `mean()` changes the result from an `xarray` object to a number so that we can easily print results in the next cell.

In [ ]:
baseline_mean = baseline_sample.mean().values
postbaseline_mean = postbaseline_sample.mean().values

In the next cell, enter the following code, and then run it to calculate the difference between the averages of the older and newer halves of the data to check for vegetation change.

The strings here start with `f` to allow code expressions like `baseline_mean` or `diff_total_mean` to be evaluated and substituted into the string when enclosed in braces. The `:.2%` in the substitution in the `Percent EVI difference` string formats the output as a percent with 2 decimal places (so the value of `diff_total_mean / baseline_mean` is approximately `-0.0159`, which is the decimal equivalent of `-1.59%`).

In [ ]:
diff_total_mean = postbaseline_mean - baseline_mean
print(f"First half EVI: {baseline_mean}")
print(f"Second half EVI: {postbaseline_mean}")
print(f"EVI difference: {diff_total_mean}")
print(f"Percent EVI difference: {(diff_total_mean / baseline_mean):.2%}")

## Conclusion

Congratulations! You have made your own vegetation change detection notebook. It is comparable to [this one](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/Real_world_examples/Vegetation_change_detection.ipynb), which may seem like a daunting task, but it is very similar to what we have done here.

You now understand how to structure a complete case study.

The additional steps in the original version will be explained in the next section.